In [1]:
import numpy as np
import random
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import spatial as sp

In [2]:
# Node and Linked List Classes

class Node:
    def __init__(self,initdata):
        self.data = initdata
        self.next = None

    def getData(self):
        return self.data

    def getNext(self):
        return self.next

    def setData(self,newdata):
        self.data = newdata

    def setNext(self,newnext):
        self.next = newnext


class OrderedList:
    def __init__(self):
        self.head = None

    def search(self,item):
        current = self.head
        found = False
        stop = False
        while current != None and not found and not stop:
            if current.getData() == item:
                found = True
            else:
                if current.getData() > item:
                    stop = True
                else:
                    current = current.getNext()

        return found
    
    def show(self):
        current = self.head
        while current != None:
            print current.getData()
            current = current.getNext()        
    
    def add(self,item):
        current = self.head
        previous = None
        stop = False
        while current != None and not stop:
            if current.getData() > item:
                stop = True
            else:
                previous = current
                current = current.getNext()

        temp = Node(item)
        if previous == None:
            temp.setNext(self.head)
            self.head = temp
        else:
            temp.setNext(current)
            previous.setNext(temp)
            
    def remove_first(self):
        self.head = self.head.getNext()
            

    def isEmpty(self):
        return self.head == None

    def size(self):
        current = self.head
        count = 0
        while current != None:
            count = count + 1
            current = current.getNext()

        return count

### Fixed variables:

In [3]:
GHZ_inner     = 20000.  # radio interno de la zona galactica habitable, años luz
GHZ_outer     = 60000.  # radio interno de la zona galactica habitable, años luz
tau_awakening = 10000.  # tiempo medio, en años, que hay que esperar para que aparezca otra CETI en la galaxia
tau_survive   = 5000.   # Tiempo medio, en años, durante el cual una CETI esta activa
D_max         = 3000.   # Maxima distancia, en años luz, a la cual una CETI puede enviar o recibir mensajes

### Simulation variables

In [4]:
# Inicializar listas de datos

# lista de CETIs alguna vez activas: dictionary
CETIs = dict()

# lista de CETIs actualmente activas: ndarray
CHATs = []

# inicializacion del tiempo: scalar
t_now = 0

# inicializacion del ID: index
ID = 0

# lista de tiempos de eventos futuros: ordered list
t_forthcoming = OrderedList()

### CASO 1

In [5]:
CHATs = [[1,2],[3,4],[5,6]]

In [61]:
case = 1

# actualizar el ID
ID = ID + 1

# sortear el lugar donde aparece dentro de la GHZ
r = np.sqrt(random.random()*GHZ_outer**2 + GHZ_inner**2)
o = random.random()*2.*np.pi
x = r * np.cos(o)  # X position on the galactic plane
y = r * np.sin(o)  # Y position on the galactic plane

# sortear el tiempo de actividad
t_active = np.random.exponential(tau_survive, 1)[0]
t_hola = t_now
t_chau = t_hola + t_active

# agregar el tiempo de desaparición a la lista de tiempos
next_event = [t_chau, ID, None, 2]
t_forthcoming.add(next_event)

# agregar la CETI a la lista histórica
#CETIs[ID].append( [(x, y, t_hola, t_chau)] )
CETIs[ID] = (x, y, t_hola, t_chau)

# sortear el tiempo de aparición de la próxima CETI
t_next_awakening = np.random.exponential(tau_awakening, 1)
next_event = [t_next_awakening, ID+1, None, 1]
t_forthcoming.add(next_event)

try:
    tree
except NameError:
    print 'No existe'
else:
    # calcular todas las CETIs dentro del radio
    query_point = [x,y]
    idx = tree.query_ball_point(query_point, r=D_max)
        
    query_point = [x,y]

    for k in idx:
        z = np.sqrt(((np.array(query_point) - np.array(CHATs[k]))**2).sum())
        
        # Agregar a t_forthcoming la lista de tiempos de contacto
        
        # 1.- Desde la recién agregada hacia las otras CETIs
        z = t_now + z
        next_event = [z, ID, k, 3]
        t_forthcoming.add(next_event)

        # 2.- Desde las otras CETIs hacia la reciés agregada
        t_hola = CETIs[2][2]
        t = z - t_hola
        next_event = [t, ID, k, 3]
        t_forthcoming.add(next_event)
   
    

# agregar la CETI a la lista de CETIs activas
# [ID, x, y, t_hola, t_chau]
CHATs.append([x, y])

# # rehacer el árbol
tree = sp.cKDTree( data=CHATs ) 

# eliminar el tiempo actual
t_forthcoming.remove_first()

In [29]:
print CETIs
print CHATs
t_forthcoming.show()

{1: (-36562.576732991511, 19936.065613787447, 0, 4322.2760697163203), 2: (-14333.079369297115, 44014.54850710382, 0, 1576.6474022002699)}
[[-36562.576732991511, 19936.065613787447]]
[array([ 2520.84758128]), 2, None, 1]
[array([ 3615.83632484]), 2, None, 1]
[4322.2760697163203, 1, None, 2]
[array([ 31835.39080199]), 3, None, 1]


### CASO 2

In [37]:
CHATs = [[1,2], [3,4]]

In [38]:
### CASO 2: desaparece una CETI
case = 2
ID = 0

# actualizar el ID
# t_now y ID son conocidos

# eliminar la CETI a la lista de CETIs activas
# [ID, x, y, t_hola, t_chau]
CHATs.pop(ID)

# rehacer el árbol
tree = sp.cKDTree( data=CHATs ) 

# eliminar el tiempo actual
t_forthcoming.remove_first()

### CASO 3

In [ ]:
### CASO 3: se produce un contacto: CETI B comienza a ver a CETI A

# eliminar la CETI a la lista de CETIs activas
# [ID, x, y, t_hola, t_chau]
ID = 1
t_chau = t_now

CETIs[ID] = [(x, y, t_hola, t_chau)]

# rehacer el árbol
M = np.column_stack([x, y])
tree = sp.cKDTree( data=M ) 


# eliminar el tiempo actual
t_next.remove_first()

### CASO 4

In [47]:
### CASO 4: se produce una baja de contacto: CETI A deja de ver a CETI B




In [39]:
acc = np.array([False]*Nstars)    # True:active communicating civilization, False: inactive/inhabited star
t_a = np.zeros(Nstars)  # awakning
t_b = np.zeros(Nstars)  # blackout
N_listening = np.zeros(Nstars) # Number of CETI listening
list_listening = np.zeros(Nstars) # list of CETI listening

In [40]:
t = 0.  # initializea
tau = 5000 #yr, mean lifetime of a ETI
lambd = 50000.  #yr, mean time until next CETI appear
D = 5000.  # light year, maximum distance of detectable signals
tmax = 1000000.  # maximo tiempo que dura la simulación

In [41]:
acc[21] = True
t = 0.
ts = []
t_last_CETI = 0.

In [42]:
def update_awakening(t, t_s_min_idx, t_b_min_idx):
    """Returns the next time at which a new CETI starts transmiting and listening
        and updates the lists.
    """
    
    global t_last_CETI
    global t_start
    global acc
    
    t_last_CETI = t
    i = np.random.choice(range(Nstars))
    t_start[i] = t
    t_s[i] = t + D
    acc[i] = True
    print 'wide awake!', t
    
    
def FCS():
    """Returns the list of the distances from active CETIs and the First Communication Surfaces"""
    
def LCS():
    """Returns the list of the distances from active CETIs and the Last Communication Surfaces"""

def update_blackout(t, t_s_min_idx, t_b_min_idx):
    """Returns the next time at which a CETI ends transmiting and listening"""
    global t_end
    
    t_end[t_b_min_idx] = t
    acc[t_b_min_idx] = False

    print 'blackout', t
    
options = {0 : update_awakening, 1 : update_sphere, 2 : update_blackout}

In [33]:
def Next_awakening():
    t_on_next = np.random.exponential(lambd, 1)
    t_a_min = t_last_CETI + t_on_next
    return(t_a_min)        

def Next_MaxReach():
    T_s = np.ma.array(t_s, mask=~acc)
    t_s_min_idx = np.ma.where(T_s == T_s.min())
    t_s_min = t_s[t_s_min_idx]
    return(t_s_min)

def Next_Blackout():
    T_b = np.ma.array(t_b, mask=~acc)
    t_b_min_idx = np.ma.where(T_b == T_b.min())
    t_b_min = t_b[t_b_min_idx]
    return(t_b_min)

Test individual functions:

In [38]:
t_a_min = Next_awakening()
t_a_min

array([ 63507.84045855])

In [35]:
Next_MaxReach()

array([ 0.])

In [36]:
Next_Blackout()

array([ 0.,  0.])

In [37]:
t = 0.
ts = []
t_last_CETI = 0.

while (t<tmax):
    
    # sortear el tiempo hasta que aparece la próxima CETI
    t_a_min = Next_awakening()
        
    # buscar el proximo tiempo en que se alcanza la esfera
    # completar la máscara con la lista de pares en contacto causal        
    t_s_min = Next_MaxReach()
    
    # buscar el proximo tiempo de desaparicion de una CETI activa
    # completar la máscara con la lista de pares en contacto causal    
    t_b_min = Next_Blackout()
        
#     mins = np.array([t_a_min, t_s_min, t_b_min])    
#     filt = mins > 0.
#     mn = np.ma.array(mins, mask=~filt)    
#     mn_idx = np.ma.where(mn == mn.min())[0][0]  # case to be considered for the next step
        
#     t = t + mins[mn_idx]
#     ts.append(t)
    
#     # Update parameters
#     options[mn_idx](t, t_s_min_idx, t_b_min_idx)    

KeyboardInterrupt: 